In [1]:
class Color:
    """定義顏色常數"""
    RED = True
    BLACK = False

class RBNode:
    """紅黑樹節點類別"""
    def __init__(self, data):
        self.data = data           # 節點存儲的數據
        self.color = Color.RED     # 新節點預設為紅色
        self.parent = None         # 父節點
        self.left = None           # 左子節點
        self.right = None          # 右子節點
    
    def is_red(self):
        """判斷節點是否為紅色"""
        return self.color == Color.RED
    
    def is_black(self):
        """判斷節點是否為黑色"""
        return self.color == Color.BLACK

class RedBlackTree:
    """紅黑樹類別"""
    def __init__(self):
        self.NIL = RBNode(None)    # 哨兵節點，代表所有的NULL葉子
        self.NIL.color = Color.BLACK
        self.root = self.NIL       # 樹根初始化為NIL
    
    def left_rotate(self, x):
        """
        左旋轉操作
        
        旋轉前：      旋轉後：
           x             y
          / \           / \
         a   y   →     x   c
            / \       / \
           b   c     a   b
        """
        y = x.right            # y是x的右子節點
        x.right = y.left       # 將y的左子樹變成x的右子樹
        
        if y.left != self.NIL:
            y.left.parent = x   # 更新y左子樹的父節點
        
        y.parent = x.parent     # 將x的父節點設為y的父節點
        
        # 更新x父節點的子節點指針
        if x.parent == self.NIL:
            self.root = y       # 如果x是根節點，y成為新根
        elif x == x.parent.left:
            x.parent.left = y   # x是左子節點
        else:
            x.parent.right = y  # x是右子節點
        
        y.left = x              # 將x設為y的左子節點
        x.parent = y            # 更新x的父節點
    
    def right_rotate(self, y):
        """
        右旋轉操作
        
        旋轉前：      旋轉後：
           y             x
          / \           / \
         x   c   →     a   y
        / \               / \
       a   b             b   c
        """
        x = y.left             # x是y的左子節點
        y.left = x.right       # 將x的右子樹變成y的左子樹
        
        if x.right != self.NIL:
            x.right.parent = y  # 更新x右子樹的父節點
        
        x.parent = y.parent     # 將y的父節點設為x的父節點
        
        # 更新y父節點的子節點指針
        if y.parent == self.NIL:
            self.root = x       # 如果y是根節點，x成為新根
        elif y == y.parent.left:
            y.parent.left = x   # y是左子節點
        else:
            y.parent.right = x  # y是右子節點
        
        x.right = y             # 將y設為x的右子節點
        y.parent = x            # 更新y的父節點
    
    def insert(self, data):
        """插入新節點"""
        # 創建新節點
        new_node = RBNode(data)
        new_node.left = self.NIL
        new_node.right = self.NIL
        
        # 執行標準BST插入
        parent = self.NIL
        current = self.root
        
        # 找到插入位置
        while current != self.NIL:
            parent = current
            if new_node.data < current.data:
                current = current.left
            else:
                current = current.right
        
        new_node.parent = parent
        
        # 確定新節點是左子還是右子
        if parent == self.NIL:
            self.root = new_node    # 樹是空的
        elif new_node.data < parent.data:
            parent.left = new_node
        else:
            parent.right = new_node
        
        # 修復紅黑樹性質
        self.insert_fixup(new_node)
    
    def insert_fixup(self, node):
        """修復插入後可能違反的紅黑樹性質"""
        while node.parent.is_red():  # 只有父節點是紅色時才需要修復
            if node.parent == node.parent.parent.left:  # 父節點是祖父的左子
                uncle = node.parent.parent.right        # 叔叔節點
                
                if uncle.is_red():  # 情況1：叔叔是紅色
                    node.parent.color = Color.BLACK
                    uncle.color = Color.BLACK
                    node.parent.parent.color = Color.RED
                    node = node.parent.parent
                else:
                    if node == node.parent.right:  # 情況2：節點是右子
                        node = node.parent
                        self.left_rotate(node)
                    
                    # 情況3：節點是左子
                    node.parent.color = Color.BLACK
                    node.parent.parent.color = Color.RED
                    self.right_rotate(node.parent.parent)
            else:  # 父節點是祖父的右子（對稱情況）
                uncle = node.parent.parent.left
                
                if uncle.is_red():  # 情況1：叔叔是紅色
                    node.parent.color = Color.BLACK
                    uncle.color = Color.BLACK
                    node.parent.parent.color = Color.RED
                    node = node.parent.parent
                else:
                    if node == node.parent.left:  # 情況2：節點是左子
                        node = node.parent
                        self.right_rotate(node)
                    
                    # 情況3：節點是右子
                    node.parent.color = Color.BLACK
                    node.parent.parent.color = Color.RED
                    self.left_rotate(node.parent.parent)
        
        self.root.color = Color.BLACK  # 根節點永遠是黑色
    
    def search(self, data):
        """搜尋節點"""
        current = self.root
        while current != self.NIL and current.data != data:
            if data < current.data:
                current = current.left
            else:
                current = current.right
        return current if current != self.NIL else None
    
    def inorder_walk(self, node=None, result=None):
        """中序遍歷"""
        if result is None:
            result = []
        if node is None:
            node = self.root
        
        if node != self.NIL:
            self.inorder_walk(node.left, result)
            color = "紅" if node.is_red() else "黑"
            result.append(f"{node.data}({color})")
            self.inorder_walk(node.right, result)
        
        return result
    
    def print_tree(self):
        """打印樹的結構（簡化版）"""
        def print_helper(node, indent="", last=True):
            if node != self.NIL:
                print(indent, end="")
                if last:
                    print("└──", end="")
                    indent += "    "
                else:
                    print("├──", end="")
                    indent += "│   "
                
                color = "紅" if node.is_red() else "黑"
                print(f"{node.data}({color})")
                
                print_helper(node.left, indent, False)
                print_helper(node.right, indent, True)
        
        if self.root == self.NIL:
            print("空樹")
        else:
            print_helper(self.root)

# 示範和測試
if __name__ == "__main__":
    print("=== 紅黑樹示範 ===\n")
    
    # 創建紅黑樹
    rbt = RedBlackTree()
    
    # 插入一些數據
    data_list = [10, 5, 15, 3, 7, 12, 18, 1, 6, 8, 11, 13, 16, 20]
    
    print("插入順序:", data_list)
    print("\n逐步插入過程：")
    
    for i, data in enumerate(data_list):
        rbt.insert(data)
        print(f"\n插入 {data} 後：")
        print("樹結構：")
        rbt.print_tree()
        print("中序遍歷：", rbt.inorder_walk())
        
        if i < 3:  # 只顯示前幾步的詳細過程
            input("按 Enter 繼續...")
    
    print("\n=== 最終結果 ===")
    print("樹結構：")
    rbt.print_tree()
    print("\n中序遍歷：", rbt.inorder_walk())
    
    # 測試搜尋
    print("\n=== 搜尋測試 ===")
    search_values = [7, 25, 13]
    for val in search_values:
        result = rbt.search(val)
        if result:
            color = "紅" if result.is_red() else "黑"
            print(f"找到 {val}，顏色：{color}")
        else:
            print(f"未找到 {val}")

=== 紅黑樹示範 ===

插入順序: [10, 5, 15, 3, 7, 12, 18, 1, 6, 8, 11, 13, 16, 20]

逐步插入過程：

插入 10 後：
樹結構：
└──10(黑)
中序遍歷： ['10(黑)']

插入 5 後：
樹結構：
└──10(黑)
    ├──5(紅)
中序遍歷： ['5(紅)', '10(黑)']

插入 15 後：
樹結構：
└──10(黑)
    ├──5(紅)
    └──15(紅)
中序遍歷： ['5(紅)', '10(黑)', '15(紅)']

插入 3 後：
樹結構：
└──10(黑)
    ├──5(黑)
    │   ├──3(紅)
    └──15(黑)
中序遍歷： ['3(紅)', '5(黑)', '10(黑)', '15(黑)']

插入 7 後：
樹結構：
└──10(黑)
    ├──5(黑)
    │   ├──3(紅)
    │   └──7(紅)
    └──15(黑)
中序遍歷： ['3(紅)', '5(黑)', '7(紅)', '10(黑)', '15(黑)']

插入 12 後：
樹結構：
└──10(黑)
    ├──5(黑)
    │   ├──3(紅)
    │   └──7(紅)
    └──15(黑)
        ├──12(紅)
中序遍歷： ['3(紅)', '5(黑)', '7(紅)', '10(黑)', '12(紅)', '15(黑)']

插入 18 後：
樹結構：
└──10(黑)
    ├──5(黑)
    │   ├──3(紅)
    │   └──7(紅)
    └──15(黑)
        ├──12(紅)
        └──18(紅)
中序遍歷： ['3(紅)', '5(黑)', '7(紅)', '10(黑)', '12(紅)', '15(黑)', '18(紅)']

插入 1 後：
樹結構：
└──10(黑)
    ├──5(紅)
    │   ├──3(黑)
    │   │   ├──1(紅)
    │   └──7(黑)
    └──15(黑)
        ├──12(紅)
        └──18(紅)
中序遍歷： ['1(紅)', '3(黑)', '5(紅)', '